# 📘 Chapter 5 — Support Vector Machines (SVM)

---


### 1) What an SVM Is Trying to Do (Linear Case)

We have a binary classification problem. A linear classifier draws a hyperplane:

$$\mathbf{w}^\top \mathbf{x} + b = 0$$

The **sign** of $\mathbf{w}^\top \mathbf{x} + b$ decides the class.

---

### 2) The Key SVM Idea: Maximize the Margin

* The **margin** is the "street width" between the two classes
* SVM doesn't just find *any* separating hyperplane — it finds the one with the **largest margin**, because:
  * it usually generalizes better
  * it is less sensitive to small shifts / noise

**Support vectors:**
* Only a few training points end up defining the boundary
* These critical points are called **support vectors**
* Move a non-support-vector → boundary often unchanged
* Move a support vector → boundary can change

> 📝 **Key idea:** The entire model is determined by a small subset of training points — the support vectors. The rest of the data is irrelevant once training is done.

---

### 3) Hard Margin vs Soft Margin

**Hard-margin SVM:**
* Assumes perfect separability (no overlaps)
* Every point must be on the correct side with a minimum margin
* **Problem:** real data is noisy → hard-margin becomes brittle or impossible

**Soft-margin SVM:**
* Allows some points to be inside the margin, or even misclassified
* Trades off: **wide margin** vs **few violations**
* This tradeoff is controlled by $C$

---

### 4) The Hyperparameter $C$ (Most Important Practical Knob)

> Think of $C$ as **"how much we hate margin violations."**

| $C$ value | Effect | Risk |
|---|---|---|
| **Large $C$** | Strongly penalizes violations → narrow margin, fits training data closely | Overfit (sensitive to noise/outliers) |
| **Small $C$** | Tolerates violations → wide margin, more robust | Underfit (if too small) |

$C$ behaves like an **inverse regularization strength**:
* Smaller $C$ → more regularization → simpler boundary
* Larger $C$ → less regularization → more complex boundary

---

### 5) Why Feature Scaling Matters a Lot for SVMs

SVMs use **distances and dot-products** heavily.

* If one feature is 1000× larger than another, it can dominate the geometry entirely
* The optimal hyperplane becomes distorted by scale

> ✅ In practice: **always use `StandardScaler` before fitting an SVM.**
```python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

svm_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm",    LinearSVC(C=1.0, max_iter=10000))
])
```

---

### 6) Key Takeaways — Linear SVM

| Concept | Core idea |
|---|---|
| Margin | "Street width" between classes — SVM maximizes it |
| Support vectors | The few points that define the boundary |
| Hard margin | Perfect separation required — brittle on real data |
| Soft margin | Allows violations — controlled by $C$ |
| $C$ large | Narrow margin, low bias, high variance |
| $C$ small | Wide margin, high bias, low variance |
| Scaling | Always scale features before SVM |

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 1) Load Iris and make it a binary problem: setosa (0) vs versicolor (1)
iris = datasets.load_iris()
X = iris.data[:, (2, 3)]          # petal length, petal width (nice for SVM demos)
y = iris.target

mask = (y != 2)                   # drop virginica => keep classes 0 and 1
X = X[mask]
y = y[mask]

# 2) Artificially create a scaling issue (multiply one feature)
X_badscale = X.copy()
X_badscale[:, 0] *= 1000.0        # petal length becomes huge-scale vs width

X_train, X_test, y_train, y_test = train_test_split(
    X_badscale, y, test_size=0.25, random_state=42, stratify=y
)

def eval_model(model, name):
    model.fit(X_train, y_train)
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
    print(f"\n{name}")
    print("  train acc:", accuracy_score(y_train, pred_train))
    print("  test  acc:", accuracy_score(y_test, pred_test))
    if hasattr(model, "named_steps") and hasattr(model.named_steps.get("svc", None), "n_support_"):
        print("  # support vectors per class:", model.named_steps["svc"].n_support_)
        print("  total support vectors:", int(model.named_steps["svc"].n_support_.sum()))

# A) Linear SVM WITHOUT scaling (should suffer because we intentionally broke scaling)
svm_no_scaling = SVC(kernel="linear", C=1.0)
eval_model(svm_no_scaling, "Linear SVM (NO scaling), C=1.0")

# B) Linear SVM WITH scaling
svm_scaled_C1 = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(kernel="linear", C=1.0))
])
eval_model(svm_scaled_C1, "Linear SVM (WITH scaling), C=1.0")

# C) Effect of C: smaller vs larger (both scaled)
svm_scaled_smallC = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(kernel="linear", C=0.1))
])
eval_model(svm_scaled_smallC, "Linear SVM (WITH scaling), C=0.1 (more regularization)")

svm_scaled_largeC = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(kernel="linear", C=100.0))
])
eval_model(svm_scaled_largeC, "Linear SVM (WITH scaling), C=100 (less regularization)")


Linear SVM (NO scaling), C=1.0
  train acc: 1.0
  test  acc: 1.0

Linear SVM (WITH scaling), C=1.0
  train acc: 1.0
  test  acc: 1.0
  # support vectors per class: [1 1]
  total support vectors: 2

Linear SVM (WITH scaling), C=0.1 (more regularization)
  train acc: 1.0
  test  acc: 1.0
  # support vectors per class: [6 6]
  total support vectors: 12

Linear SVM (WITH scaling), C=100 (less regularization)
  train acc: 1.0
  test  acc: 1.0
  # support vectors per class: [1 1]
  total support vectors: 2


# 📘 Chapter 5 — Nonlinear SVMs + The Kernel Trick

---

## ✅ C Controls "Strictness" — Quick Lock-In

> Lower $C$ ⇒ violations penalized less ⇒ wider margin ⇒ more points end up on/inside the margin ⇒ **more support vectors**

* `C = 0.1` → 12 support vectors (tolerant, wide margin)
* `C = 1.0` → 2 support vectors (clean separation)
* `C = 100` → 2 support vectors (strict, but data already separable)

---

## B) Nonlinear SVMs + The Kernel Trick

### 1) Why We Need Kernels

A linear SVM draws a **straight boundary** in the original feature space. But many datasets are not linearly separable.

**Idea:**
* Transform features into a **higher-dimensional space** where the data becomes separable
* Then run a linear SVM there

$$\mathbf{w}^\top \phi(\mathbf{x}) + b = 0$$

---

### 2) The Kernel Trick (The Practical Magic)

In training/prediction, SVM mostly needs **dot products**:

$$\phi(\mathbf{x}_i)^\top \phi(\mathbf{x}_j)$$

A **kernel function** computes this dot product **without explicitly computing** $\phi(\mathbf{x})$:

$$K(\mathbf{x}_i, \mathbf{x}_j) = \phi(\mathbf{x}_i)^\top \phi(\mathbf{x}_j)$$

> 📝 **Key idea:** Kernels let us get nonlinear decision boundaries efficiently — we never actually transform the data into the high-dimensional space. We just compute as *if* we did.

---

### 3) Polynomial Kernel (Smooth-ish Curved Boundaries)

$$K(\mathbf{x}, \mathbf{z}) = (\gamma\, \mathbf{x}^\top \mathbf{z} + r)^d$$

In scikit-learn (`kernel="poly"`):

| Parameter | Effect |
|---|---|
| `degree` ($d$) | Higher = more complex curves |
| `coef0` ($r$) | Shifts influence of higher-order vs lower-order terms |
| `C` | Softness / regularization (same as linear case) |

> **Intuition:** polynomial kernel creates feature interactions like $x_1^2$, $x_1 x_2$, etc. — without explicitly computing them.

---

### 4) Gaussian RBF Kernel (Most Popular Default)

$$K(\mathbf{x}, \mathbf{z}) = \exp\big(-\gamma \|\mathbf{x} - \mathbf{z}\|^2\big)$$

Two key knobs: **$C$** and **$\gamma$ (gamma)**

---

### 5) What $\gamma$ Controls (Super Important)

| $\gamma$ value | Influence region | Boundary | Risk |
|---|---|---|---|
| **Small $\gamma$** | Wide — each point influences many others | Smooth, simple | Underfit |
| **Large $\gamma$** | Narrow — each point influences only nearby points | Wiggly, complex | Overfit |

> 📝 **Intuition:** $\gamma$ is like the "reach" of each training point. Small $\gamma$ = each point "votes" over a wide area. Large $\gamma$ = each point only affects its immediate neighborhood.

---

### 6) How $C$ and $\gamma$ Interact

| Combo | Effect |
|---|---|
| Large $C$ + Large $\gamma$ | Very complex boundary → **overfit** |
| Small $C$ + Small $\gamma$ | Very smooth boundary → **underfit** |
| Large $C$ + Small $\gamma$ | Strict but smooth |
| Small $C$ + Large $\gamma$ | Tolerant but locally complex |

> ✅ In practice: tune $C$ and $\gamma$ together (e.g., with `GridSearchCV`). They interact strongly.

---

### 7) Full Kernel Comparison

| Kernel | Best when | Key params |
|---|---|---|
| `linear` | Data is (nearly) linearly separable | `C` |
| `poly` | Smooth nonlinear boundary, known degree | `C`, `degree`, `coef0` |
| `rbf` | General nonlinear — **default first choice** | `C`, `gamma` |

> 📝 **Rule of thumb:** Start with RBF kernel. If data is high-dimensional and sparse, try linear first.

In [2]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X, y = make_moons(n_samples=400, noise=0.25, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

def run(C, gamma):
    model = Pipeline([
        ("scaler", StandardScaler()),
        ("svc", SVC(kernel="rbf", C=C, gamma=gamma))
    ])
    model.fit(X_train, y_train)
    train_acc = accuracy_score(y_train, model.predict(X_train))
    test_acc  = accuracy_score(y_test,  model.predict(X_test))
    sv = model.named_steps["svc"].n_support_.sum()
    print(f"C={C:<6} gamma={gamma:<6} | train={train_acc:.3f} test={test_acc:.3f} | total SV={sv}")

print("RBF SVM: effect of gamma (C fixed)")
for g in [0.01, 0.1, 1, 10]:
    run(C=1.0, gamma=g)

print("\nRBF SVM: interaction of C and gamma")
for C in [0.1, 1, 100]:
    for g in [0.1, 10]:
        run(C=C, gamma=g)

RBF SVM: effect of gamma (C fixed)
C=1.0    gamma=0.01   | train=0.810 test=0.930 | total SV=173
C=1.0    gamma=0.1    | train=0.820 test=0.950 | total SV=134
C=1.0    gamma=1      | train=0.937 test=0.930 | total SV=87
C=1.0    gamma=10     | train=0.960 test=0.900 | total SV=162

RBF SVM: interaction of C and gamma
C=0.1    gamma=0.1    | train=0.813 test=0.940 | total SV=187
C=0.1    gamma=10     | train=0.950 test=0.920 | total SV=293
C=1      gamma=0.1    | train=0.820 test=0.950 | total SV=134
C=1      gamma=10     | train=0.960 test=0.900 | total SV=162
C=100    gamma=0.1    | train=0.927 test=0.940 | total SV=84
C=100    gamma=10     | train=0.993 test=0.930 | total SV=106


# 📘 Chapter 5 — Choosing a Kernel + Tuning (C, γ, degree, coef0)

---

## ✅ γ Check Answer Lock-In

* Increasing $\gamma$ (with $C$ fixed) → boundary becomes **more local / more wiggly**
* Train accuracy rises, test accuracy can fall → **classic overfitting risk**
* Best test score usually at a **medium $\gamma$** (e.g., 0.1) — smooth enough to generalize

---

## B) Explanation

### 1) Which Kernel Should I Try First?

**Default choice: RBF kernel**
* Works well for many problems
* Models complex boundaries without hand-designing features

**Polynomial kernel**
* Good when the boundary is well-modeled by low-degree feature interactions (quadratic/cubic)
* More interpretable than RBF in some settings (`degree` = "complexity level")

> 📝 **Rule of thumb:** No reason to prefer otherwise → **start with RBF**. If you believe "interactions up to degree 2/3 are enough" → try poly.

---

### 2) What Hyperparameters Control Complexity?

**RBF SVM:**

| Parameter | Small value | Large value |
|---|---|---|
| $\gamma$ | Smooth, global influence → underfit | Very local, wiggly → **overfit** |
| $C$ | Wide margin, more violations → regularized | Strict fit to training data → **overfit** |

> Overfit combo: **large $C$ + large $\gamma$**
> Underfit combo: **small $C$ + small $\gamma$**

**Polynomial SVM:**

| Parameter | Role |
|---|---|
| `degree` ($d$) | Main complexity knob (2, 3, 4, …) |
| `coef0` ($r$) | Shifts influence of high-order vs low-order terms |
| `C` | Soft margin — same role as always |
| `gamma` | Exists, but `degree` + `coef0` are the main "feel" controls |

---

### 3) Interpreting Support Vectors (Very Practical)

* **More support vectors** → harder boundary to define (more points are close to / inside the margin)
* At extreme $\gamma$ values (very small or very large), support vector counts often rise — the model geometry becomes "awkward" in different ways:
  * Very small $\gamma$ → too smooth, many points near the margin
  * Very large $\gamma$ → too local, many points individually influential

---

### 4) Practical Tuning Workflow
```python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# RBF grid
rbf_params = {
    "svm__C":     [0.1, 1, 10, 100],
    "svm__gamma": [0.01, 0.1, 1, 10]
}

# Polynomial grid
poly_params = {
    "svm__C":      [0.1, 1, 10],
    "svm__degree": [2, 3, 4],
    "svm__coef0":  [0, 1, 5]
}

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svm",    SVC(kernel="rbf"))   # swap kernel="poly" for poly search
])

grid_search = GridSearchCV(pipe, rbf_params, cv=5, scoring="accuracy")
grid_search.fit(X_train, y_train)

print("Best params:", grid_search.best_params_)
print("Best CV score:", grid_search.best_score_)
print("Test score:", grid_search.score(X_test, y_test))
```

---

### 5) Full Tuning Reference

| Kernel | Tune first | Then tune |
|---|---|---|
| `rbf` | `C`, `gamma` | — |
| `poly` | `degree`, `C` | `coef0` |
| `linear` | `C` | — |

> ✅ Always scale features with `StandardScaler` before any SVM kernel — distances and dot products are scale-sensitive.

In [3]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Same dataset as before
X, y = make_moons(n_samples=400, noise=0.25, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC())
])

# 1) RBF grid
rbf_grid = {
    "svc__kernel": ["rbf"],
    "svc__C": [0.1, 1, 10, 100],
    "svc__gamma": [0.01, 0.1, 1, 10],
}

# 2) Poly grid (keep it modest so it runs fast)
poly_grid = {
    "svc__kernel": ["poly"],
    "svc__C": [0.1, 1, 10],
    "svc__degree": [2, 3, 4],
    "svc__coef0": [0, 1],
    "svc__gamma": ["scale"],  # good default; keeps search smaller
}

def run_grid(param_grid, name):
    gs = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
    gs.fit(X_train, y_train)

    best = gs.best_estimator_
    test_acc = best.score(X_test, y_test)
    sv_total = int(best.named_steps["svc"].n_support_.sum())

    print(f"\n{name}")
    print("  best CV score:", gs.best_score_)
    print("  best params  :", gs.best_params_)
    print("  test score   :", test_acc)
    print("  total SV     :", sv_total)

run_grid(rbf_grid, "RBF SVM")
run_grid(poly_grid, "Polynomial SVM")


RBF SVM
  best CV score: 0.9266666666666667
  best params  : {'svc__C': 100, 'svc__gamma': 1, 'svc__kernel': 'rbf'}
  test score   : 0.89
  total SV     : 45

Polynomial SVM
  best CV score: 0.9299999999999999
  best params  : {'svc__C': 10, 'svc__coef0': 1, 'svc__degree': 4, 'svc__gamma': 'scale', 'svc__kernel': 'poly'}
  test score   : 0.92
  total SV     : 53


# 📘 Chapter 5 — SVM Regression (SVR): The ε-Insensitive Tube

---

## ✅ Kernel Wiggliness Lock-In

* **RBF:** $\gamma$ mainly controls wiggliness (local vs global influence); $C$ controls strictness (soft margin)
* **Polynomial:** `degree` is the main shape complexity knob; `coef0` shifts higher-order vs lower-order term balance; $C$ controls strictness; $\gamma$ matters if not left at `scale`

---

## B) Explanation

### 1) What SVR Tries to Do

SVR is the SVM idea applied to **regression**.

Instead of separating classes, SVR fits a function $f(\mathbf{x})$ such that most points lie **inside a tube** around the prediction:

* **Tube half-width = $\varepsilon$ (epsilon)**
* Point **inside** the tube → **no penalty**
* Point **outside** the tube → penalized (grows with distance beyond $\varepsilon$)

This is the **$\varepsilon$-insensitive loss:**

| Region | Loss |
|---|---|
| $\|y - f(\mathbf{x})\| \leq \varepsilon$ | $0$ |
| $\|y - f(\mathbf{x})\| > \varepsilon$ | $\|y - f(\mathbf{x})\| - \varepsilon$ |

> 📝 **Key idea:** SVR doesn't care about errors as long as they're small enough (within the tube). This makes it robust to small noise.

---

### 2) SVR Support Vectors (Key Insight)

In SVR, the points that matter are:
* Points **on the tube boundary**
* Points **outside the tube**

These become the **support vectors**.

| $\varepsilon$ size | Points outside tube | Support vectors | Model complexity |
|---|---|---|---|
| **Large $\varepsilon$** (wide tube) | Few | Few | Simpler → more bias |
| **Small $\varepsilon$** (narrow tube) | Many | Many | More complex → more variance |

---

### 3) What $C$ Does in SVR

Same principle as classification — $C$ controls how much we punish points outside the tube:

| $C$ value | Effect | Risk |
|---|---|---|
| **Large $C$** | Strongly penalize violations → fits training points closely | Overfit |
| **Small $C$** | Tolerates violations → smoother model | Underfit |

---

### 4) RBF SVR: $\gamma$ Still Matters

Same as classification:
* Larger $\gamma$ → more local → wigglier regression curve
* Smaller $\gamma$ → more global → smoother curve

---

### 5) Scaling

> ✅ SVR (especially RBF) is very sensitive to feature scale → **always use `StandardScaler`**.

---

### 6) Full SVR Parameter Reference

| Parameter | Controls | Large value | Small value |
|---|---|---|---|
| $\varepsilon$ | Tube width | Fewer SVs, simpler, more bias | More SVs, complex, more variance |
| $C$ | Penalty for violations | Strict fit → overfit risk | Tolerant → underfit risk |
| $\gamma$ (RBF) | Local vs global influence | Wiggly → overfit | Smooth → underfit |

---

### 7) SVR vs SVM Classification — Side by Side

| | SVM Classification | SVR |
|---|---|---|
| Goal | Maximize margin between classes | Fit function within $\varepsilon$-tube |
| Support vectors | Points on/inside the margin | Points on/outside the tube |
| $C$ role | Penalize margin violations | Penalize tube violations |
| $\varepsilon$ role | — | Controls tube width (tolerance) |
| Kernels | Linear, Poly, RBF | Same options |

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# Prefer modern sklearn RMSE function if available; otherwise fall back to sqrt(MSE)
try:
    from sklearn.metrics import root_mean_squared_error
    def rmse(y_true, y_pred):
        return root_mean_squared_error(y_true, y_pred)
except Exception:
    def rmse(y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))

# 1D nonlinear regression dataset
rng = np.random.RandomState(42)
X = np.linspace(0, 6, 300).reshape(-1, 1)
y = np.sin(X).ravel() + 0.25 * rng.randn(len(X))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

def eval_svr(C, gamma, epsilon):
    model = Pipeline([
        ("scaler", StandardScaler()),
        ("svr", SVR(kernel="rbf", C=C, gamma=gamma, epsilon=epsilon))
    ])
    model.fit(X_train, y_train)

    pred_train = model.predict(X_train)
    pred_test  = model.predict(X_test)

    rmse_train = rmse(y_train, pred_train)
    rmse_test  = rmse(y_test, pred_test)

    n_sv = int(model.named_steps["svr"].support_.shape[0])
    print(f"C={C:<6} gamma={gamma:<5} eps={epsilon:<4} | RMSE train={rmse_train:.3f} test={rmse_test:.3f} | SV={n_sv}")

print("Effect of epsilon (C and gamma fixed)")
for eps in [0.01, 0.1, 0.3, 0.7]:
    eval_svr(C=10, gamma=1, epsilon=eps)

print("\nInteraction: epsilon with different C")
for C in [0.1, 1, 100]:
    eval_svr(C=C, gamma=1, epsilon=0.1)

Effect of epsilon (C and gamma fixed)
C=10     gamma=1     eps=0.01 | RMSE train=0.255 test=0.217 | SV=220
C=10     gamma=1     eps=0.1  | RMSE train=0.254 test=0.211 | SV=153
C=10     gamma=1     eps=0.3  | RMSE train=0.257 test=0.206 | SV=57
C=10     gamma=1     eps=0.7  | RMSE train=0.321 test=0.282 | SV=6

Interaction: epsilon with different C
C=0.1    gamma=1     eps=0.1  | RMSE train=0.263 test=0.206 | SV=160
C=1      gamma=1     eps=0.1  | RMSE train=0.255 test=0.206 | SV=153
C=100    gamma=1     eps=0.1  | RMSE train=0.253 test=0.217 | SV=155


# 📘 Chapter 5 — Under the Hood of SVMs

---

## ✅ ε Lock-In

* Larger $\varepsilon$ = wider tube → **fewer support vectors**, more bias, more tolerance
* Smaller $\varepsilon$ = narrower tube → **more support vectors**, tighter fit, more variance

---

## B) Explanation

### 1) Soft-Margin Linear SVM = "Large Margin + Penalty for Violations"

A soft-margin linear SVM chooses $\mathbf{w}, b$ by balancing:
* **Large margin** (keep $\|\mathbf{w}\|$ small)
* **Few margin violations** (penalize points inside the margin / misclassified)

$$\min_{\mathbf{w}, b}\;\;\frac{1}{2}\|\mathbf{w}\|^2 + C\sum_{i=1}^{m}\max(0,\; 1 - y_i(\mathbf{w}^\top \mathbf{x}_i + b))$$

Where:
* $y_i \in \{-1, +1\}$
* $\max(0, 1 - y_i f(\mathbf{x}_i))$ = the **hinge loss**
* $C$ controls the tradeoff (strictness)

---

### 2) Hinge Loss (What It "Charges" You For)

Define the margin value:

$$t_i = y_i(\mathbf{w}^\top \mathbf{x}_i + b)$$

| $t_i$ value | Situation | Loss |
|---|---|---|
| $t_i \geq 1$ | Correctly classified, outside margin | $0$ |
| $0 < t_i < 1$ | Correct side but inside margin | $> 0$ |
| $t_i \leq 0$ | Misclassified | $\geq 1$ |

> 📝 **Key idea:** Only points **on/inside the margin** affect the objective. Points far away contribute nothing.

---

### 3) Why Support Vectors Are Special

In the final solution:
* Points **far from the boundary** → zero hinge loss → don't push on the boundary
* Points **on the margin**, **inside the margin**, or **misclassified** → non-zero hinge loss → **these become support vectors**

> The boundary is entirely defined by support vectors. Remove a non-support-vector and nothing changes. Move a support vector and the boundary shifts.

---

### 4) Dual Form (Why Kernels Work)

In the dual formulation, the decision function becomes:

$$f(\mathbf{x}) = \sum_{i \in SV} \alpha_i y_i K(\mathbf{x}_i, \mathbf{x}) + b$$

**Key takeaways:**
* It's a **sum over support vectors only** — non-SVs have $\alpha_i = 0$
* Data enters only via **kernel values** $K(\mathbf{x}_i, \mathbf{x})$
* That's why we get nonlinear boundaries **without explicitly mapping features**

For a linear kernel $K(\mathbf{x}_i, \mathbf{x}) = \mathbf{x}_i^\top \mathbf{x}$, we can recover:

$$\mathbf{w} = \sum_{i \in SV} \alpha_i y_i \mathbf{x}_i$$

---

### 5) Primal vs Dual — Side by Side

| | Primal | Dual |
|---|---|---|
| Variables | $\mathbf{w}, b$ | $\alpha_i$ (one per training point) |
| Decision function | $\mathbf{w}^\top \mathbf{x} + b$ | $\sum_{i \in SV} \alpha_i y_i K(\mathbf{x}_i, \mathbf{x}) + b$ |
| Non-SVs | Zero hinge loss | $\alpha_i = 0$ |
| Kernels | Hard to apply | **Natural fit** |
| Efficient when | Many instances, few features | Few instances, many features |

---

### 6) Full Under-the-Hood Summary

$$\underbrace{\frac{1}{2}\|\mathbf{w}\|^2}_{\text{maximize margin}} + \underbrace{C \sum_i \max(0, 1 - y_i f(\mathbf{x}_i))}_{\text{penalize violations}}$$

* **Hinge loss** = the "charge" for being on the wrong side of the margin
* **Support vectors** = the only points with non-zero $\alpha_i$ → they alone define the boundary
* **Kernel trick** = replace $\mathbf{x}_i^\top \mathbf{x}$ with $K(\mathbf{x}_i, \mathbf{x})$ in the dual → nonlinear boundaries for free

In [5]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

X, y01 = make_moons(n_samples=300, noise=0.25, random_state=42)

# Convert labels {0,1} -> {-1,+1} for hinge-loss style math
y = np.where(y01 == 1, 1, -1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

model = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(kernel="linear", C=1.0))
])

model.fit(X_train, y_train)
svc = model.named_steps["svc"]

# Support vectors and dual coefficients
SV = svc.support_vectors_                 # shape (n_SV, n_features)
alpha_y = svc.dual_coef_.ravel()          # shape (n_SV,), equals alpha_i * y_i
b = float(svc.intercept_[0])

# Reconstruct w from dual: w = sum_i (alpha_i * y_i * x_i)
w_dual = alpha_y @ SV                     # shape (n_features,)

print("n_support_ per class:", svc.n_support_, " total:", int(np.sum(svc.n_support_)))
print("w from dual:", w_dual)
print("b:", b)

# Pick one test point and compare three equivalent computations
x = model.named_steps["scaler"].transform(X_test[:1])[0]   # IMPORTANT: compare in scaled space

# 1) sklearn decision function (uses internal representation)
sk_dec = float(svc.decision_function([x])[0])

# 2) primal form: w^T x + b
primal = float(w_dual @ x + b)

# 3) dual form: sum_i (alpha_i y_i <x_i, x>) + b
dual = float(alpha_y @ (SV @ x) + b)

print("\nOne test point comparison")
print("  sklearn decision:", sk_dec)
print("  primal  w^T x + b:", primal)
print("  dual sum over SV :", dual)
print("  abs differences  :", abs(sk_dec - primal), abs(sk_dec - dual))

n_support_ per class: [36 37]  total: 73
w from dual: [ 0.758042   -1.46055609]
b: -0.011180260556931257

One test point comparison
  sklearn decision: 0.9580109549633407
  primal  w^T x + b: 0.9580109549633428
  dual sum over SV : 0.9580109549633424
  abs differences  : 2.1094237467877974e-15 1.7763568394002505e-15


# 📘 Chapter 5 — LinearSVC vs SVC(kernel="linear") vs SGDClassifier(hinge)

---

## ✅ Hinge Loss Lock-In

* If $t_i = y_i f(\mathbf{x}_i) \geq 1$ → hinge loss = **0** → point does **not** become a support vector
* Only points on/inside the margin ($t_i < 1$) have non-zero $\alpha_i$ → only they define the boundary

---

## B) Explanation

### 1) Three Ways to Train a Linear SVM-Like Classifier

**A) `SVC(kernel="linear")`**
* Full SVM class with linear kernel — uses classic kernel SVM machinery
* ✅ Gives `support_vectors_`, easy to switch to RBF/poly later
* ❌ Slow / memory-heavy on large $m$ (kernel-SVM methods don't scale well)

> **Use when:** small/medium datasets, or when you want support vectors / might switch to kernels

---

**B) `LinearSVC`**
* Specialized for linear SVMs — no kernel trick
* Optimized for large datasets and high-dimensional sparse features (text, TF-IDF)
* ✅ Much faster than `SVC(kernel="linear")` on large $m$
* ❌ No kernel trick, typically no `support_vectors_`

> **Use when:** linear boundary is enough + lots of samples and/or many features

---

**C) `SGDClassifier(loss="hinge")`**
* Trains a linear classifier with hinge loss via stochastic gradient descent
* Not the exact same optimizer as classic SVM solvers, but targets a similar objective
* ✅ Works for very large-scale / streaming data, supports `partial_fit`
* ❌ More sensitive to tuning (learning rate, epochs, regularization), results can be noisier

> **Use when:** massive dataset / streaming / need online updates

---

### 2) Practical Decision Rule

| Situation | Best choice |
|---|---|
| Want nonlinear boundary | `SVC(kernel="rbf")` or `SVC(kernel="poly")` |
| Linear + big/sparse dataset (e.g. text) | `LinearSVC` |
| Linear + huge/streaming data | `SGDClassifier(loss="hinge")` |
| Want support vectors / smaller dataset | `SVC(kernel="linear")` |

---

### 3) Under the Hood — Why They Differ

| | `SVC(kernel="linear")` | `LinearSVC` | `SGDClassifier(hinge)` |
|---|---|---|---|
| Solver | libsvm (kernel SVM) | liblinear | SGD |
| Scales with $m$ | ❌ Poor | ✅ Good | ✅ Excellent |
| Kernel trick | ✅ Yes | ❌ No | ❌ No |
| `support_vectors_` | ✅ Yes | ❌ No | ❌ No |
| Online learning | ❌ No | ❌ No | ✅ `partial_fit` |
| Tuning sensitivity | Low | Low | **Higher** |

---

### 4) Regularization Note

All three regularize — but the parameter names differ:

| Classifier | Regularization param | Direction |
|---|---|---|
| `SVC` | `C` | Larger $C$ = less regularization |
| `LinearSVC` | `C` | Larger $C$ = less regularization |
| `SGDClassifier` | `alpha` | Larger `alpha` = more regularization |

> 📝 `SGDClassifier` uses `alpha` (like Ridge/Lasso), not `C`. Conceptually $\alpha \approx \frac{1}{C}$.

In [6]:
import time
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier

# A moderately sized dataset
X, y = make_classification(
    n_samples=12000, n_features=40, n_informative=15, n_redundant=5,
    class_sep=1.2, flip_y=0.03, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

def eval_model(name, model):
    t0 = time.perf_counter()
    model.fit(X_train, y_train)
    fit_s = time.perf_counter() - t0

    pred = model.predict(X_test)
    acc = accuracy_score(y_test, pred)

    extra = ""
    # If it's an SVC in a pipeline, we can access support vectors
    if hasattr(model, "named_steps") and "svc" in model.named_steps:
        svc = model.named_steps["svc"]
        if hasattr(svc, "support_"):
            extra = f" | #SV={len(svc.support_)}"

    print(f"{name:28s} fit={fit_s:.3f}s | test acc={acc:.3f}{extra}")

# 1) SVC linear (kernel SVM machinery)
svc_linear = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(kernel="linear", C=1.0))
])

# 2) LinearSVC (large-scale linear SVM)
linearsvc = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LinearSVC(C=1.0, max_iter=20000))
])

# 3) SGDClassifier hinge (linear SVM-like)
# Rough mapping: alpha ≈ 1 / (C * n_samples) (useful heuristic)
C = 1.0
alpha = 1.0 / (C * len(X_train))
sgd_hinge = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", SGDClassifier(loss="hinge", alpha=alpha, max_iter=2000, tol=1e-3, random_state=42))
])

eval_model("SVC(kernel='linear')", svc_linear)
eval_model("LinearSVC", linearsvc)
eval_model("SGDClassifier(loss='hinge')", sgd_hinge)

SVC(kernel='linear')         fit=21.858s | test acc=0.843 | #SV=3596
LinearSVC                    fit=0.104s | test acc=0.843
SGDClassifier(loss='hinge')  fit=0.610s | test acc=0.818


# 📘 Chapter 5 — Nonlinear SVM: Polynomial Features vs Kernels

---

## ✅ LinearSVC Check Lock-In

For 2M samples + 100K sparse TF-IDF features → **`LinearSVC`**
* Scales well with large $m$ (liblinear solver)
* Built for sparse, high-dimensional data
* `SVC(kernel="linear")` would be prohibitively slow; `SGDClassifier` needs more tuning

---

## B) Explanation

### 1) Two Ways to Get a Nonlinear Decision Boundary

**Method 1: Explicit Feature Expansion (Feature Engineering)**

Transform input features into a richer set, then fit a linear model there.

Example — polynomial features:
* Original: $(x_1, x_2)$
* Add: $x_1^2, x_2^2, x_1 x_2, \dots$
* A **linear SVM in the expanded space** → **nonlinear boundary in the original space**

| | Pros | Cons |
|---|---|---|
| Explicit expansion | Simple mental model | Feature explosion → slow |
| Kernel trick | Powerful, no explicit mapping | Kernel SVMs slow on huge $m$ |

---

**Method 2: Kernels (Kernel Trick)**

Instead of explicitly computing $\phi(\mathbf{x})$, the SVM uses:

$$K(\mathbf{x}, \mathbf{z}) = \phi(\mathbf{x})^\top \phi(\mathbf{z})$$

Nonlinear boundaries — without explicitly building all features.

---

### 2) Polynomial Features vs Polynomial Kernel (Important Comparison)

| Approach | How it works | Notes |
|---|---|---|
| `PolynomialFeatures` + `LinearSVC` | Explicitly creates polynomial terms → linear model on expanded features | Transparent, but feature explosion at high degree |
| `SVC(kernel="poly")` | Implicitly computes polynomial dot products via kernel | No explicit feature matrix → more memory efficient |

> 📝 They often behave similarly but are **not always identical** — different solvers, scaling conventions, and regularization details.

---

### 3) When to Use Which

| Situation | Best approach |
|---|---|
| Small dataset, low degree | `PolynomialFeatures` + `LinearSVC` (interpretable) |
| Medium dataset, unknown degree | `SVC(kernel="poly")` with grid search on `degree` |
| General nonlinear, medium dataset | `SVC(kernel="rbf")` — default first choice |
| Very large dataset, nonlinear | Consider neural networks or ensemble methods |

---

### 4) Full Nonlinearity Strategy Ladder

$$\text{Linear SVM} \xrightarrow{\text{not enough}} \text{Poly Features + LinearSVC} \xrightarrow{\text{feature explosion}} \text{Kernel SVM (poly/RBF)}$$

> 📝 **Key idea:** The kernel trick lets you work in an implicitly high-dimensional space **without the memory/compute cost** of explicitly building that space. This is the core power of kernel methods.

In [7]:
import time
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score

X, y = make_moons(n_samples=2000, noise=0.25, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

def eval_model(name, model):
    t0 = time.perf_counter()
    model.fit(X_train, y_train)
    fit_s = time.perf_counter() - t0
    acc = accuracy_score(y_test, model.predict(X_test))
    print(f"{name:35s} fit={fit_s:.3f}s | test acc={acc:.3f}")

# 1) Linear SVM on raw features
linear_raw = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LinearSVC(C=1.0, max_iter=20000))
])

# 2) Explicit polynomial features + linear SVM
poly_features = Pipeline([
    ("poly", PolynomialFeatures(degree=3, include_bias=False)),
    ("scaler", StandardScaler()),
    ("clf", LinearSVC(C=1.0, max_iter=20000))
])

# 3) Polynomial kernel SVM
poly_kernel = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(kernel="poly", degree=3, C=1.0, coef0=1))
])

eval_model("LinearSVC (raw features)", linear_raw)
eval_model("PolyFeatures(deg=3)+LinearSVC", poly_features)
eval_model("SVC(poly kernel, deg=3)", poly_kernel)

# Also show how many features the polynomial expansion creates
Xt = PolynomialFeatures(degree=3, include_bias=False).fit_transform(X_train[:1])
print("\n# features after degree-3 polynomial expansion:", Xt.shape[1])

LinearSVC (raw features)            fit=0.010s | test acc=0.884
PolyFeatures(deg=3)+LinearSVC       fit=0.017s | test acc=0.928
SVC(poly kernel, deg=3)             fit=0.066s | test acc=0.926

# features after degree-3 polynomial expansion: 9


# 📘 Chapter 5 — RBF SVM Tuning Playbook (C, γ)

---

## 1) Always Scale First

Use `StandardScaler()` before fitting any SVM.

> Without scaling, $\gamma$ behaves unpredictably — distances get distorted by feature scale differences.

---

## 2) Mental Model: What Each Knob Does

**$\gamma$ (gamma) = wiggliness / locality**

| $\gamma$ value | Influence region | Boundary | Risk |
|---|---|---|---|
| Small | Wide — global | Smooth | Underfit |
| Large | Narrow — local | Very wiggly | Overfit |

**$C$ = strictness about violations**

| $C$ value | Margin | Violations | Risk |
|---|---|---|---|
| Small | Wide | Many allowed | Underfit |
| Large | Narrow | Few tolerated | Overfit |

---

## 3) Practical Search Strategy

Start with a **coarse grid on powers of 10**:

$$C \in \{0.1,\ 1,\ 10,\ 100\}$$
$$\gamma \in \{0.01,\ 0.1,\ 1,\ 10\}$$

Pick the best combination by **cross-validation**.
```python
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(kernel="rbf"))
])

param_grid = {
    "svm__C":     [0.1, 1, 10, 100],
    "svm__gamma": [0.01, 0.1, 1, 10]
}

grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring="accuracy")
grid_search.fit(X_train, y_train)

print("Best params:", grid_search.best_params_)
print("Test score:", grid_search.score(X_test, y_test))
```

---

## 4) Diagnose Quickly: Train vs Test Pattern

| Pattern | Diagnosis | Fix |
|---|---|---|
| Train low + Test low | **Underfitting** | ↑ $C$ and/or ↑ $\gamma$ |
| Train high + Test noticeably lower | **Overfitting** | ↓ $C$ and/or ↓ $\gamma$ |
| Train high + Test high | ✅ Good fit | Done |

---

## 5) Full Tuning Loop (Mental Model)
```
Scale features
    ↓
Coarse grid search (powers of 10)
    ↓
Diagnose: underfit or overfit?
    ↓
Refine grid around best region
    ↓
Evaluate once on test set
```

> 📝 **Key idea:** $C$ and $\gamma$ interact — always tune them **together**, not independently. The best combo is rarely at an extreme of either.

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(kernel="rbf"))
])

param_grid = {
    "svc__C": [0.1, 1, 10, 100],
    "svc__gamma": [0.01, 0.1, 1, 10],
}

gs = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
gs.fit(X_train, y_train)

print(gs.best_params_, gs.best_score_)
print("test:", gs.best_estimator_.score(X_test, y_test))

{'svc__C': 1, 'svc__gamma': 1} 0.9486666666666667
test: 0.934


# 📘 Chapter 5 — Support Vector Machines: Full Recap

---

## A) Where to Read (Pinpoint References)

| Section | Keywords |
|---|---|
| Large-Margin Classification | `Large margin` |
| Soft Margin Classification | `C` |
| Nonlinear SVM Classification | `Nonlinear` |
| Polynomial Features | `polynomial`, `features` |
| Kernel Trick | `Kernel Trick` |
| Polynomial Kernel | `degree`, `coef0` |
| Gaussian RBF Kernel | `gamma`, `RBF` |
| Support Vector Regression | `SVR`, `ε-insensitive` |
| Under the Hood | `hinge loss`, `dual` |
| Large-Scale Linear SVM | `LinearSVC`, `SGDClassifier` |

---

## B) Chapter 5 Recap

### 1) Big Idea: Large-Margin Classification

A linear SVM finds a separating hyperplane:

$$f(\mathbf{x}) = \mathbf{w}^\top \mathbf{x} + b$$

Instead of just separating the classes, SVM **maximizes the margin** (the "street width" between classes).

> Larger margins usually generalize better — less sensitive to small data noise.

---

### 2) Support Vectors

Only some training points determine the boundary: the **support vectors**.

* Points **far from the boundary** → usually don't affect the solution
* Points **on/inside the margin** (or misclassified) → **critical → support vectors**

---

### 3) Soft Margin + The Role of $C$

Real datasets aren't perfectly separable → we allow **margin violations**.

| $C$ value | Effect | Risk |
|---|---|---|
| **Large $C$** | Penalize violations strongly → strict fit | Overfit |
| **Small $C$** | Tolerate violations → wider margin | Underfit |

> Smaller $C$ often increases the number of support vectors — more points fall inside the margin.

---

### 4) Feature Scaling Is (Almost) Mandatory

SVMs depend heavily on **dot-products / distances**. If features have different scales, one feature can dominate.

> ✅ Standard practice: `StandardScaler` → SVM (always inside a `Pipeline`)

---

### 5) Nonlinear SVMs: Two Routes

**Route A: Polynomial Features + Linear Model**
* Explicitly create new features (e.g., $x_1^2$, $x_1 x_2$, …), then fit a linear SVM
* ✅ Straightforward idea | ❌ Feature space can blow up

**Route B: Kernels (Kernel Trick)**
* Don't explicitly build $\phi(\mathbf{x})$ — use a kernel to compute:

$$K(\mathbf{x}, \mathbf{z}) = \phi(\mathbf{x})^\top \phi(\mathbf{z})$$

* Result: nonlinear boundary *as if* you mapped into a higher-dimensional space

---

### 6) Polynomial Kernel (Key Knobs)

Produces smooth nonlinear boundaries based on polynomial interactions.

| Parameter | Effect |
|---|---|
| `degree` | Higher = more complex curve |
| `coef0` | Shifts influence of higher-order vs lower-order terms |
| `C` | Soft margin strictness (same as always) |

---

### 7) RBF (Gaussian) Kernel — Most Common Default

$$K(\mathbf{x}, \mathbf{z}) = \exp(-\gamma \|\mathbf{x} - \mathbf{z}\|^2)$$

| Parameter | Small value | Large value |
|---|---|---|
| $\gamma$ | Wide influence → smooth → **underfit** | Local influence → wiggly → **overfit** |
| $C$ | More regularization → tolerant | Stricter fit → **overfit** |

**Common patterns:**
* Overfit: **large $C$ + large $\gamma$**
* Underfit: **small $C$ + small $\gamma$**

---

### 8) Simple Tuning Loop (Chapter-Level Playbook)

1. **Scale features** (`StandardScaler`)
2. **Coarse grid** (powers of 10):

$$C \in \{0.1,\ 1,\ 10,\ 100\}, \quad \gamma \in \{0.01,\ 0.1,\ 1,\ 10\}$$

3. **Diagnose:**

| Pattern | Diagnosis | Fix |
|---|---|---|
| Train low + Test low | Underfit | ↑ $C$ and/or ↑ $\gamma$ |
| Train high + Test lower | Overfit | ↓ $C$ and/or ↓ $\gamma$ |

---

### 9) SVR: The ε-Insensitive Tube

Goal: fit a function where points **inside a tube of width $\varepsilon$** incur **zero loss**.

| Parameter | Large value | Small value |
|---|---|---|
| $\varepsilon$ | Wider tube → fewer SVs → simpler | Tighter tube → more SVs → complex |
| $C$ | Fits strictly | Smooth, tolerant |
| $\gamma$ (RBF) | Wiggly curve | Smooth curve |

---

### 10) Under the Hood: Hinge Loss + Dual View

Soft-margin SVM uses **hinge loss** — points correctly classified beyond the margin contribute **0 loss**.

Decision function = **sum over support vectors only**:

$$f(\mathbf{x}) = \sum_{i \in SV} \alpha_i y_i K(\mathbf{x}_i, \mathbf{x}) + b$$

> This is why kernels work: the model only needs kernel values $K(\cdot, \cdot)$, not explicit $\phi(\mathbf{x})$.

---

### 11) Linear SVM Implementations: When to Use Which

| Classifier | Best when | Key advantage |
|---|---|---|
| `SVC(kernel="linear")` | Small/medium datasets | Exposes support vectors explicitly |
| `LinearSVC` | Large datasets, sparse features (TF-IDF) | Optimized for large $m$ |
| `SGDClassifier(loss="hinge")` | Very large / streaming / online learning | Supports `partial_fit` |

# 📘 Chapter 5 — SVM Exercises: Q&A

---

**Q1. What is the fundamental idea behind Support Vector Machines?**

An SVM tries to find a decision boundary (hyperplane) that separates classes while **maximizing the margin** (the distance to the nearest training points). This "largest-margin" choice tends to generalize better. With a **soft margin**, it also allows some violations while balancing them against a wider margin using the hyperparameter $C$.

---

**Q2. What is a support vector?**

A support vector is a training instance that lies **on the margin boundary or inside the margin** (including misclassified points), meaning it directly influences the position of the decision boundary. Points far from the boundary typically have **no effect** on the final model.

---

**Q3. Why is it important to scale the inputs when using SVMs?**

SVMs rely heavily on **dot products and distances** (especially with kernels like RBF). If features are on very different scales, large-scale features dominate the geometry — distorting margins and kernel similarities. Scaling (e.g., standardization) makes each feature contribute more fairly and usually improves both performance and stability.

---

**Q4. Can an SVM classifier output a confidence score when it classifies an instance? What about a probability?**

* **Confidence score:** Yes — an SVM naturally outputs a **decision score** (signed distance to the decision boundary), which serves as a confidence-like ranking.
* **Probability:** Not natively. True probabilities require extra **probability calibration** (e.g., Platt scaling or isotonic regression), which adds a fitting step and may trade some speed for probabilistic outputs.

---

**Q5. Should you use the primal or the dual form to train a model on millions of instances and hundreds of features?**

With **millions of instances** and only **hundreds of features**, prefer the **primal** (or primal-like) approach — the dual typically scales poorly with the number of training instances. In practice this means using efficient **linear SVM solvers** (`LinearSVC`) or **SGD-based hinge-loss training** (`SGDClassifier`) rather than a full kernelized dual formulation.

---

**Q6. You trained an RBF SVM and it underfits. Should you increase or decrease γ? What about C?**

Underfitting = model too simple → **increase both**:
* **Increase $\gamma$** → more flexible/local boundary
* **Increase $C$** → reduce regularization, penalize violations more strongly

⚠️ Adjust gradually — too-large $\gamma$ or $C$ can quickly cause overfitting.

---

**Q7. How should you set the QP parameters (H, f, A, b) to solve the soft-margin linear SVM using an off-the-shelf QP solver?**

Let the variable vector be $\boldsymbol{\theta} = [\mathbf{w};\ b;\ \boldsymbol{\xi}]$ where $\mathbf{w} \in \mathbb{R}^n$, $b \in \mathbb{R}$, and $\boldsymbol{\xi} \in \mathbb{R}^m$.

| QP param | Setting |
|---|---|
| $\mathbf{H}$ | Block matrix: identity on the $\mathbf{w}$-block, zeros elsewhere → quadratic term = $\frac{1}{2}\|\mathbf{w}\|^2$ |
| $\mathbf{f}$ | Zeros for $\mathbf{w}$ and $b$; $C$ for each slack $\xi_i$ → linear term = $C\sum \xi_i$ |
| $\mathbf{A}, \mathbf{b}$ | Encode constraints $y_i(\mathbf{w}^\top \mathbf{x}_i + b) \geq 1 - \xi_i$ and $\xi_i \geq 0$ into the solver's inequality form |

> The exact sign/stacking depends on whether your solver expects $A\boldsymbol{\theta} \leq \mathbf{b}$ or $A\boldsymbol{\theta} \geq \mathbf{b}$.


---

**Q8. Train a LinearSVC, SVC, and SGDClassifier on the same linearly separable dataset. Can you get them to produce roughly the same model?**

Yes — use the same scaling, set all three to optimize a **linear decision boundary**:
* `SVC(kernel="linear")`
* `LinearSVC` (as-is)
* `SGDClassifier(loss="hinge")`

Tune their regularization so strengths are comparable (conceptually: $C$ in SVC/LinearSVC ↔ $\frac{1}{\alpha \cdot n}$ in SGDClassifier). For SGD, ensure enough iterations and a stable learning-rate schedule.

If done correctly, all three should yield **very similar decision boundaries and accuracy** — with `LinearSVC` and `SGDClassifier` usually training much faster than `SVC` on large datasets.

---

**Q9. Train an SVM classifier on the MNIST dataset. What accuracy can you reach?**

* **Linear SVM (one-vs-rest):** typically reaches low-to-mid **90%** accuracy fairly quickly
* **RBF-kernel SVM** with careful tuning of $C$ and $\gamma$: can often reach **97–98%**, but training becomes significantly slower and memory-heavier

> ✅ Practical approach: use a coarse-to-fine grid search on a smaller validation set first, then confirm on the full test set. Scaling is essential — always `StandardScaler` before fitting.

---

**Q10. Train an SVM regressor on the California housing dataset.**

Workflow:
1. **Standardize features** (`StandardScaler`) — SVR is very sensitive to scale
2. **Baseline:** `LinearSVR` — fast, establishes a floor
3. **Nonlinear:** `SVR(kernel="rbf")` — tune $C$, $\gamma$, and $\varepsilon$ via cross-validation
4. **Evaluate** with RMSE (or MAE)

Expected outcome: RBF SVR captures nonlinearities better than the linear baseline when tuned well, though it is slower and more sensitive to hyperparameters and scaling.

| Model | Speed | Nonlinearity | Sensitivity |
|---|---|---|---|
| `LinearSVR` | Fast | ❌ No | Low |
| `SVR(kernel="rbf")` | Slower | ✅ Yes | High — tune carefully |

Q8) LinearSVC vs SVC(linear) vs SGDClassifier on a linearly separable dataset

In [9]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

# 1) Make a (nearly) linearly separable dataset
X, y = make_classification(
    n_samples=6000,
    n_features=20,
    n_informative=10,
    n_redundant=0,
    n_clusters_per_class=1,
    class_sep=2.5,      # higher => more separable
    flip_y=0.0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

# 2) Scale (important for SVMs)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

# 3) Train the three linear models
C = 1000.0  # large C ~ close to hard-margin if separable

svc = SVC(kernel="linear", C=C)
svc.fit(X_train_s, y_train)

linearsvc = LinearSVC(C=C, max_iter=20000)
linearsvc.fit(X_train_s, y_train)

# Heuristic mapping: alpha ≈ 1 / (C * n_samples)
alpha = 1.0 / (C * len(X_train_s))
sgd = SGDClassifier(
    loss="hinge",
    alpha=alpha,
    max_iter=5000,
    tol=1e-4,
    random_state=42
)
sgd.fit(X_train_s, y_train)

# 4) Evaluate accuracy
def eval_acc(name, model):
    pred = model.predict(X_test_s)
    print(f"{name:26s} test acc = {accuracy_score(y_test, pred):.4f}")

eval_acc("SVC(kernel='linear')", svc)
eval_acc("LinearSVC", linearsvc)
eval_acc("SGDClassifier(hinge)", sgd)

# 5) Compare the learned hyperplanes (direction of w)
# Note: all trained on the SAME scaled data => w is directly comparable.
w_svc, b_svc = svc.coef_.ravel(), float(svc.intercept_[0])
w_lsv, b_lsv = linearsvc.coef_.ravel(), float(linearsvc.intercept_[0])
w_sgd, b_sgd = sgd.coef_.ravel(), float(sgd.intercept_[0])

def cos_sim(a, b):
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

print("\nCosine similarity between weight vectors (±1 means same direction):")
print("  SVC vs LinearSVC:", cos_sim(w_svc, w_lsv))
print("  SVC vs SGD      :", cos_sim(w_svc, w_sgd))
print("  LinearSVC vs SGD:", cos_sim(w_lsv, w_sgd))

print("\nIntercepts (b):")
print("  SVC      :", b_svc)
print("  LinearSVC:", b_lsv)
print("  SGD      :", b_sgd)

print("\n# Support vectors (SVC only):", len(svc.support_))

SVC(kernel='linear')       test acc = 0.9973
LinearSVC                  test acc = 0.9973
SGDClassifier(hinge)       test acc = 0.9973

Cosine similarity between weight vectors (±1 means same direction):
  SVC vs LinearSVC: 0.9869088995683594
  SVC vs SGD      : 0.9757334343153079
  LinearSVC vs SGD: 0.9921400725507815

Intercepts (b):
  SVC      : 0.722504011649743
  LinearSVC: 0.3255123367277458
  SGD      : 78.85635406986505

# Support vectors (SVC only): 19


Q9) SVM classifier on MNIST with One-vs-Rest (OvR) + quick tuning

This does OvR explicitly (as requested). RBF SVM on full MNIST can be slow; the code tunes on a smaller subset first.

In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

# 1) Load MNIST (70k x 784)
X, y = fetch_openml("mnist_784", version=1, as_frame=False, return_X_y=True)
y = y.astype(np.int64)

# Optional: scale pixels to [0,1] first (helps numerics)
X = X / 255.0

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 2) Baseline: OvR Linear SVM (fast)
linear_ovr = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", OneVsRestClassifier(LinearSVC(C=1.0, max_iter=20000)))
])
linear_ovr.fit(X_train, y_train)
pred = linear_ovr.predict(X_test)
print("OvR LinearSVC test acc:", accuracy_score(y_test, pred))

# 3) RBF SVM (OvR) with tuning on a small subset
# Subsample to speed up CV tuning
rng = np.random.RandomState(42)
subset_size = 12000
idx = rng.choice(len(X_train), size=subset_size, replace=False)
X_sub, y_sub = X_train[idx], y_train[idx]

rbf_ovr = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", OneVsRestClassifier(SVC(kernel="rbf")))
])

param_grid = {
    "clf__estimator__C": [1, 10, 100],
    "clf__estimator__gamma": [0.01, 0.03, 0.1],
}

gs = GridSearchCV(rbf_ovr, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
gs.fit(X_sub, y_sub)

print("\nBest CV score:", gs.best_score_)
print("Best params  :", gs.best_params_)

# 4) Evaluate best RBF OvR model on the full test set
best_rbf = gs.best_estimator_
pred = best_rbf.predict(X_test)
print("OvR RBF-SVC test acc:", accuracy_score(y_test, pred))

Q10) SVM regressor on California Housing (LinearSVR baseline + RBF SVR on subset)

Full RBF SVR on the entire dataset can be heavy; this code trains RBF SVR on a subset (still the correct SVR method).

In [ ]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR, SVR
from sklearn.metrics import mean_squared_error

# RMSE helper compatible with newer sklearn
def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# 1) Load dataset
data = fetch_california_housing()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2) Baseline: LinearSVR (fast)
linear_svr = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", LinearSVR(C=1.0, epsilon=0.1, random_state=42, max_iter=20000))
])
linear_svr.fit(X_train, y_train)
pred = linear_svr.predict(X_test)
print("LinearSVR RMSE:", rmse(y_test, pred))

# 3) RBF SVR (slower): tune on a subset for practicality
rng = np.random.RandomState(42)
subset_size = 12000
idx = rng.choice(len(X_train), size=subset_size, replace=False)
X_sub, y_sub = X_train[idx], y_train[idx]

rbf_svr = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", SVR(kernel="rbf"))
])

param_grid = {
    "svr__C": [1, 10, 100],
    "svr__gamma": [0.01, 0.1, 1.0],
    "svr__epsilon": [0.05, 0.1, 0.2],
}

gs = GridSearchCV(rbf_svr, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
gs.fit(X_sub, y_sub)

print("\nBest CV RMSE (neg):", gs.best_score_)
print("Best params       :", gs.best_params_)

best_rbf_svr = gs.best_estimator_
pred = best_rbf_svr.predict(X_test)
print("RBF SVR RMSE:", rmse(y_test, pred))